In [4]:
from selenium import webdriver
import os
import selenium
import time
from PIL import Image
import io
import requests 

In [6]:
DRIVER_PATH = "webdriver\geckodriver-v0.30.0-win64\geckodriver.exe"

In [27]:
def scroll_to_end(driver):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
def image_url_function(string, max_links, driver):
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"
    driver.get(search_url.format(q=string))
    image_urls = set()
    image_count = 0
    results_start = 0

    while image_count < max_links:
        scroll_to_end(driver)
        thumbnail_results = driver.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")

        for img in thumbnail_results[results_start:number_results]:
            try:
                img.click()
                time.sleep(1)
            except Exception:
                continue
            
            actual_images = driver.find_elements_by_css_selector("img.n3VNCb")
            for actual_image in actual_images:
                if actual_image.get_attribute("src") and "http" in actual_image.get_attribute("src"):
                    image_urls.add(actual_image.get_attribute("src"))
            
            image_count = len(image_urls)
            if image_count >= max_links:
                print(f"Found: {image_count} image links")
                break
        else:
            print("Found:", image_count, "looking for more image links ...")
            time.sleep(30)
            return
            
            load_more_button = driver.find_element_by_css_selector(".mye4qd")
            
            if load_more_button:
                driver.execute_script("document.querySelector('.mye4qd').click();")

        results_start = len(thumbnail_results)

    return image_urls

def save_images(folder_path,file_name,url):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - COULD NOT DOWNLOAD {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        
        file_path = os.path.join(folder_path, file_name)
        
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SAVED - {url} - AT: {file_path}")
    except Exception as e:
        print(f"ERROR - COULD NOT SAVE {url} - {e}")




In [31]:
driver = webdriver.Firefox(executable_path=DRIVER_PATH)
search_names = ["goku", "vegeta"]
images_path = f'img'

for name in search_names:
    if not os.path.isdir(images_path):
        os.mkdir(images_path)
    path = os.path.join(images_path,name)
    if not os.path.isdir(path):
        os.mkdir(path)

    driver.get('https://google.com')
    search_box = driver.find_element_by_css_selector('input.gLFyf')
    search_box.send_keys(name)
    links = image_url_function(name,20,driver)

    for i, link in enumerate(links):
        file_name = f"{i:06}.jpg"
        save_images(path,file_name,link)
driver.quit()